In [103]:
# Imports 

import numpy as np 
import matplotlib.pyplot as plt 
import matplotlib.ticker as mticker 
import os 
import itertools 
import mesa_reader as mr 


import utils.load_data as load_data
import utils.plotting.profile_plotting as profile_plotting 
import utils.plotting.history_plotting as history_plotting 
import utils.plotting.HR_diagram_plotting as HR_diagram_plotting
import utils.config.ui_options as ui_options 
import utils.config.plot_options as plot_options
import utils.config.stellar_evolution_data as stellar_evolution_data 
import utils.config.physical_constants as physical_constants 
import utils.helpers as helpers 


# Automatically reload modules 
%load_ext autoreload 
%autoreload 2 

# Make matplotlib plots open in a separate interactive window 
%matplotlib qt 



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
# To do: 






# Research questions 

# Look in a 6 and 10 solar mass star, the carbon and nitrogen abundances after helium fusion has ignited. 
# At some point, nitrogen should plummet while carbon stays at the same level. 
# C+N should go down. Normally is constant because you're just turning C into N, but now we're depleting the N as well. 
# High temp: C+N rises because you're creating C and there is no N 
# There should be a thin layer that used to be in the core but isn't anymore. C+N will drop. 



# Additional data, file structures 

# 7-9 solar masses: Fuses carbon but not neon, forms an oxygen–neon–magnesium (ONeMg or ONe) white dwarf (https://en.wikipedia.org/wiki/White_dwarf)
# Find ages to stop simulation in order to get each flowchart point (goal: middle of the stage)
# Add support for high mass stars (separate flowchart entirely?) 
# 3.0 M_sun is missing C+O WD phase 
# 1.75 M_sun is missing C+O WD phase 
# Add modelnum_start and modelnum_end for all substages 
# Git ignore: Remove old Jupyter notebooks from the repo, add MESA files 





# Flowchart 

# Add "we are are" showing currently selected mass and model number 
# Add transparent/gray boxes where the star doesn't achieve those stages with explanation why it skips those stages. I.e.: "never gets hot enough to fuse helium" 
# Re-think colors used in flowchart boxes/colors used to represent the substages 
########### Add spectral types to mass selection ("0.1-0.3" become "0.1-0.3 (M1-K3)", etc) 
# entire flowchart should always be visible. Use 3 levels of highlight (1: selected, 2: unselected but available for comparison, 3: unavailable for comparison)





# HR diagram 

# Run build_combo_cache() command once, save data to a CSV, and then load the CSV 
# Add transparent tracks of available but un-selected substages for comparison 
# Finish incorporting Mode1 and Mode2 colored tracks of HR diagram paths based on the Paint pictures I made earlier 
# HR diagram we are here point color 





# History plots 

# Add an option for history plot to be either scaled linearly with time or to evenly space the substages, 
# to make it easier to see the interesting properties that happen all near the end of the star's life
# How to deal with helium ignition: give an option called is_instantaneous=True which overrides the need for a model_start and model_end. 
# Instead, it uses the model_example and plots a LINE at that point rather than an axhspan, and the even spacing ignores it. 
# Colored regions on history plot that correspond to each stage and get highlighted if its selected 
# History plot we are here vertical line color/linestyle (match HR diagram?)



# HR Diagram and history plot 

# "We are here" vertical line on history plot + "we are here" yellow label on HR diagram should match colors/styles and both be labeled 



# Fusion history plot 

# Add separate lines for CNO cycle fusion and PP fusion 
# Coordinate colors with the colors used by profile plots 




# Temp grad profile plot 

# Radiative vs adiabatic temp gradients: theoretical = dashed? 





# How to make  marimo notebook available on github 

# Take URL to this notebook on github, which is: 
# https://github.com/johnmomberg/Gayley_Stellar_Evolution_Textbook/blob/main/stellar_evolution_marimo_script.py 
# Replace the https://github.com/ part with https://marimo.app/github.com/ , which gives: 
# https://marimo.app/github.com/johnmomberg/Gayley_Stellar_Evolution_Textbook/blob/main/stellar_evolution_marimo_script.py 



# Misc 

# Check colors of all plots 
# Check linestyles and linewidths 
# Maybe go for densly dashed rather than just "dashed"? ls=(0,(5,1)) 
# Rewrite modelnum labels function on history plot 
# ###################Add "No selection" option to tabs, so that in mode 1, you can zoom out and view the entire history plot 
    # (Rather than automatically crop to a specific stage)



In [258]:
# Testing profile or history plots  

history = load_data.load_history(stellar_evolution_data.data_folder/"M=1.0")
profile = load_data.load_profile(stellar_evolution_data.data_folder/"M=1.0", modelnum=300, history=history) 

# _ = profile_plotting.ProfilePlot.temp(profile, history=history) 
# _ = profile_plotting.ProfilePlot.degeneracy(profile, history=history) 
# _ = profile_plotting.ProfilePlot.fusion(profile, history=history) 
# _ = profile_plotting.ProfilePlot.mu(profile, history=history) 
# _ = profile_plotting.ProfilePlot.composition(profile, history=history) 
# _ = history_plotting.HistoryPlot.mass(history) 

for modelnum in history.model_numbers_available: 
    print(history.star_age[modelnum-1])
    print(history.age_strings[modelnum-1])
    print("\n")  




1e-05
1e-05 years


0.45497190750010674
0.455 years


2510.5960734982905
2.511 k years


110735.24027571104
110.7 k years


5684788.479823111
5.685 M years


21199892.584104393
21.2 M years


43020069.65187602
43.02 M years


73895630.93890288
73.9 M years


4790159781.894712
4.79 G years


5245928072.347926
5.246 G years


9878231859.774849
9.878 G years


11463105253.945868
11.46 G years


11635404481.56518
11.64 G years


11933594887.477743
11.93 G years


12054440235.921297
12.05 G years


12122479427.975267
12.12 G years


12166216244.018866
12.17 G years


12197938315.6237
12.2 G years


12219992562.103527
12.22 G years


12237227798.195677
12.24 G years


12250812854.856676
12.25 G years


12261887860.535652
12.26 G years


12271021543.379004
12.27 G years


12279953340.267078
12.28 G years


12289292395.187252
12.29 G years


12296954289.234005
12.297 G years


12303360548.976011
12.303 G years


12308794509.797674
12.309 G years


12313510536.323336
12.314 G years


1231761558

0.0


In [ ]:
# import math 

# def round_sigfigs(x, num_sigfigs): 
#     return round(x, num_sigfigs - int(math.floor(math.log10(abs(x)))) - 1)



# age_previous = 12476323636.879574
# age_current =  12476326110.007242
# age_next =     12476326198.957787 

# num_sigfigs = np.max([len(str(age).replace(".", "")) for age in [age_previous, age_current, age_next]]) 

# # Initialize 
# rounded_previous = round_sigfigs(age_previous, num_sigfigs) 
# rounded_current = round_sigfigs(age_current, num_sigfigs) 
# rounded_next = round_sigfigs(age_next, num_sigfigs) 

# # Keep looping until rounding has gone too far, then take the previous iteratin 
# while str(rounded_current) != str(rounded_previous) and str(rounded_current) != str(rounded_next): 

#     age_previous = rounded_previous 
#     age_current = rounded_current 
#     age_next = rounded_next 

#     rounded_previous = round_sigfigs(age_previous, num_sigfigs) 
#     rounded_current = round_sigfigs(age_current, num_sigfigs) 
#     rounded_next = round_sigfigs(age_next, num_sigfigs) 

#     print(num_sigfigs)
#     print(age_previous)
#     print(age_current)  
#     print(age_next) 
#     print('\n')

#     num_sigfigs-=1 




17
12476323636.879574
12476326110.007242
12476326198.957787


16
12476323636.879574
12476326110.007242
12476326198.957787


15
12476323636.87957
12476326110.00724
12476326198.95779


14
12476323636.8796
12476326110.0072
12476326198.9578


13
12476323636.88
12476326110.007
12476326198.958


12
12476323636.88
12476326110.01
12476326198.96


11
12476323636.9
12476326110.0
12476326199.0


10
12476323637.0
12476326110.0
12476326199.0


9
12476323640.0
12476326110.0
12476326200.0


8
12476323600.0
12476326100.0
12476326200.0




882.0

In [26]:
folder = stellar_evolution_data.data_folder/"Mass=1.75_models=every5" 
history = load_data.load_history(folder) 



plt.figure(figsize=(10,8))
plt.plot(history.star_age, history.he_core_mass, label="Helium", color="tab:green")
plt.plot(history.star_age, history.c_core_mass, label="Carbon", color="tab:red")
plt.plot(history.star_age, history.o_core_mass, label="Oxygen", color="tab:brown")
plt.plot(history.star_age, history.fe_core_mass, label="Iron", color="black")
plt.ylabel("Mass")
plt.legend() 

plt.figure(figsize=(10,8))
plt.plot(history.star_age, history.he_core_radius, label="Helium", color="tab:green")
plt.plot(history.star_age, history.c_core_radius, label="Carbon", color="tab:red")
plt.plot(history.star_age, history.o_core_radius, label="Oxygen", color="tab:brown")
plt.plot(history.star_age, history.fe_core_radius, label="Iron", color="black")
plt.ylabel("Radius")
plt.legend() 




In [ ]:
# Test gradient rectangle 


import numpy as np
import matplotlib.pyplot as plt
from matplotlib.transforms import blended_transform_factory

fig, ax = plt.subplots(figsize=(8, 4))

# Example background plot
ax.plot([0, 1, 2, 3], [0, 1, 0, 1], color='gray')

# Define where your gradient bar goes
xmin, xmax = 1, 2
height = 0.1
y_bottom = 1.0  # top of the plot
trans = blended_transform_factory(ax.transData, ax.transAxes)

# Create a gradient array from white → color → white
n = 256
color = np.array([1.0, 0.3, 0.3])  # your base color (e.g. red)
white = np.ones(3)
gradient = np.linspace(0, 1, n)
rgb = np.outer(1 - np.abs(gradient * 2 - 1), color) + np.outer(np.abs(gradient * 2 - 1), white)

# rgb shape (n,3) → (1,n,3) for imshow
gradient_img = rgb[np.newaxis, :, :]

# Display gradient as image in data coordinates
ax.imshow(
    gradient_img,
    extent=(xmin, xmax, y_bottom, y_bottom + height),
    transform=trans,
    aspect='auto',
    clip_on=False,
    zorder=3,
)

ax.set_xlim(0, 3)
ax.set_ylim(0, 1.5)
plt.show()


In [6]:
# Look for boundaries between substages 


folder = stellar_evolution_data.data_folder/"Mass=1.75_models=every5" 
history = load_data.load_history(folder)
print(history.model_numbers_available)
modelnum = 11425  


# folder = stellar_evolution_data.data_folder/"MESA-Web_Job_10282555412" 
# history = load_data.load_history(folder)
# print(history.model_numbers_available)
# modelnum = 9800 


# HR diagram 
test = HR_diagram_plotting.HRDiagram()
test.add_path(history, label=history.initial_mass_string)
plt.scatter(10**history.log_Teff[modelnum-1], 10**history.log_L[modelnum-1], color="red", ec="black", zorder=100)  
plt.legend() 

# Composition and fusion vs age 
# _ = history_plotting.HistoryPlot.composition(history, modelnum_now=modelnum)  
# _ = history_plotting.HistoryPlot.fusion(history, modelnum_now=modelnum)  
_ = history_plotting.HistoryPlot.radius(history, modelnum_now=modelnum)  
# _ = history_plotting.HistoryPlot.mass(history, modelnum_now=modelnum)  

# Interior composition 
if modelnum in history.model_numbers_available: 
    profile = load_data.load_profile(folder, modelnum=modelnum, history=history) 
    profile_plotting.ProfilePlot.composition(profile, history=history) 
    # profile_plotting.ProfilePlot.convection(profile, history=history) 
    profile_plotting.ProfilePlot.fusion(profile, history=history) 


# Print info 
print(f"model = {modelnum}") 
print(f"age = {history.star_age[modelnum-1]}") 







[    1   140   220   235   265   285   305   389  1000  2000  3000  4000
  5000  7000  6000  8000  8763  8802  9000  9800 10000 11225 11230 11235
 11240 11245 11250 11255 11260 11265 11270 11275 11280 11285 11290 11295
 11300 11305 11310 11315 11320 11325 11330 11335 11340 11345 11350 11355
 11360 11365 11370 11375 11380 11385 11390 11395 11400 11405 11410 11415
 11420 11425 11430 11435 11440 11445 11450 11455 11460 11465 11470 11475
 11480 11485 11490 11495 11500 11505 11510 11515 11520 11525 11530 11535
 11540 11545 11550 11555 11560 11565 11570 11575 11580 11585 11590 11595
 11600 11605 11610 11615 11620 11625 11630 11635 11640 11645 11650 11655
 11660 11665 11670 11675 11680 11685 11690 11695 11700 11705 11710 11715
 11720 11725 11730 11735 11740 11745 11750 11755 11760 11765 11770 11775
 11780 11785 11790 11795 11800 11805 11810 11815 11820 11825 11830 11835
 11840 11845 11850 11855 11860 11865 11870 11875 11880 11885 11890 11895
 11900 11905 11910 11915 11920 11925 11930 11935 11

In [ ]:
# 0.2 M_sun (represents 0.1 - 0.3) 
# Fully convective 

# Middle of hayashi track (already available in stellar_evolution_data.data_folder/"M=0.2") 
# model = 150
# age = 733779.8346512254 

# Middle of MS (already available in stellar_evolution_data.data_folder/"M=0.2")** 
# model = 273 
# age = 462063191253 

# Middle of He WD (already available in stellar_evolution_data.data_folder/"M=0.2")
# model = 1200
# age = 1026553681667.4374

In [ ]:
# 0.4 M_sun (represents 0.3 - 0.5) 
# No henyey track, not fully convective, never fuses helium  

# Middle of Hayashi track (already available in stellar_evolution_data.data_folder/"M=0.4") 
# model = 200
# age = 1657716.2443819284  

# Middle of MS (already available in stellar_evolution_data.data_folder/"M=0.4") 
# model = 309
# age = 72844865022.73999 

# Subgiant (already available in stellar_evolution_data.data_folder/"M=0.4") 
# 450? 

# Red giant (already available in stellar_evolution_data.data_folder/"M=0.4") 
# 3000? 

# Helium white dwarf (already available in stellar_evolution_data.data_folder/"M=0.4") 
# 5159 




In [ ]:
# 1.0 M_sun (represents 0.5 - 1.5) 

# Hayashi 
# start = 1 
# model = 150
# end = 202 

# Henyey 
# start = 202
# model = 220
# end = 240  

# MS 
# start = 240
# model = 296
# end = 330

# Subgiant 
# start = 330 
# model = 389 
# end = 415 

# Red giant 
# start = 415 
# model = 5000
# end = 9500 

# Helium flash 
# start = 9500 
# model = 9700 
# end = 10500 

# He MS 
# start = 10500 
# model = 10650
# end = 10950 

# AGB 
# start = 10950 
# model = 12300 
# end = 13600 

# On the way to Helium white dwarf 
# start = 13600 
# model = 14300
# end = 14300 

In [ ]:
# 1.75 M_sun (represents 1.5 - 2) 
# Hertzsprung gap, but helium flash? 

# Hayashi 
# start = 1 
# model = 140 
# end = 200 

# Henyey 
# start = 200 
# model = 235 
# end =  250 

# MS 
# start = 250 
# model = 285 
# end = 340 

# Hertzsprung gap  
# start = 340 
# model = 389 
# end = 420 

# Red giant 
# start = 420 
# model = 1000 
# end = 8763

# Helium flash 
# start =  8763
# model = 8802
# end = 9000 

# He MS 
# start = 9000 
# model = 9800 
# end =  10000

# AGB 
# start =  10000
# model =  11425
# end =  13635

# On the way to Helium white dwarf 
# start =  
# model = 
# end = 

In [ ]:
# 3 M_sun (represents 2 - 6) 

# Hayashi (already available in stellar_evolution_data.data_folder/"M=3.0") 
# 150 

# Henyey (already available in stellar_evolution_data.data_folder/"M=3.0")** 
# model = 225
# age = 2063039.3885299314 

# MS (already available in stellar_evolution_data.data_folder/"M=3.0") 
# 300 

# Hertzsprung Gap (already available in stellar_evolution_data.data_folder/"M=3.0")** 
# model = 363 
# age = 322089285.1496673

# Red giant (already available in stellar_evolution_data.data_folder/"M=3.0") 
# 400 

# Helium stable ignition 

# Helium main sequence (already available in stellar_evolution_data.data_folder/"M=3.0") 
# 650?  

# AGB (already available in stellar_evolution_data.data_folder/"M=3.0") 
# 1700

# C+O white dwarf 
# Need to extend age!! 



In [196]:
# Calculate moment of inertia and M*R**2 

m = 10.0 
history = load_data.load_history(stellar_evolution_data.data_folder/f"M={m}")


# Calculate moment of inertia  
def calc_inertia(profile):

    # Sort arrays so they start at center of star, and convert to CGS units 
    ind_sort = np.argsort(profile.mass)
    mass_coord_sorted_g = profile.mass[ind_sort] * physical_constants.M_sun 
    radius_coord_sorted_cm = profile.radius[ind_sort] * physical_constants.R_sun
    
    # I = integral of 2/3 * r**2 over the mass coordinate (2/3 factor because its the I of a spherical shell)
    inertia = np.trapz(2/3* radius_coord_sorted_cm**2, x=mass_coord_sorted_g)
    
    return inertia 


ages = [] 
inertias = [] 
m_rsquareds = [] 
for modelnum in history.model_numbers_available: 
    print(f"{modelnum} / {np.max(history.model_numbers_available)}")
    profile = load_data.load_profile(stellar_evolution_data.data_folder/f"M={m}", modelnum=modelnum, history=history) 
    ages.append(history.star_age[modelnum-1]) 
    inertias.append(calc_inertia(profile))
    m_rsquareds.append(history.star_mass[modelnum-1]*physical_constants.M_sun * ((10**history.log_R[modelnum-1])*physical_constants.R_sun)**2)




# Plot moment of inertia and MR^2 
plt.figure(figsize=(15, 8))
plt.plot(ages, m_rsquareds, label="MR^2", lw=3) 
plt.plot(ages, inertias, label="Moment of inertia", lw=3) 
plt.yscale("log")
plt.xlabel("Age (years)")
plt.ylabel("grams * cm^2") 
title = f"Moment of Inertia vs time of a {m} M_sun star"
plt.title(title)
plt.grid(alpha=0.5) 
plt.legend() 
plt.savefig(title + ".jpg")


# Plot ratio of I to MR^2 to get the f-constant in I=fMR^2 
plt.figure(figsize=(15, 8))
plt.plot(ages, np.array(inertias)/np.array(m_rsquareds), lw=3) 
plt.xlabel("Age (years)")  
title = f"f vs time for a {m} M_sun star"
plt.title(title)
plt.grid(alpha=0.5) 
plt.savefig(title + ".jpg")








1 / 2550
50 / 2550
100 / 2550
150 / 2550
200 / 2550
250 / 2550
272 / 2550
300 / 2550
322 / 2550
350 / 2550
354 / 2550
400 / 2550
412 / 2550
419 / 2550
450 / 2550
494 / 2550
500 / 2550
550 / 2550
600 / 2550
650 / 2550
700 / 2550
750 / 2550
800 / 2550
850 / 2550
900 / 2550
950 / 2550
1000 / 2550
1050 / 2550
1100 / 2550
1150 / 2550
1200 / 2550
1250 / 2550
1300 / 2550
1350 / 2550
1400 / 2550
1450 / 2550
1500 / 2550
1550 / 2550
1600 / 2550
1650 / 2550
1700 / 2550
1750 / 2550
1800 / 2550
1850 / 2550
1900 / 2550
1950 / 2550
2000 / 2550
2050 / 2550
2100 / 2550
2150 / 2550
2200 / 2550
2250 / 2550
2300 / 2550
2350 / 2550
2400 / 2550
2450 / 2550
2500 / 2550
2550 / 2550


In [ ]:
# Calculate moment of inertia and M*R**2 

m = 10.0 
history = load_data.load_history(stellar_evolution_data.data_folder/f"M={m}")


# Calculate internal energy 
def calc_internal_energy(profile):

    # Sort arrays so they start at center of star, and convert to CGS units 
    ind_sort = np.argsort(profile.mass) 
    mass_coord_sorted_g = profile.mass[ind_sort] * physical_constants.M_sun 
    temp_sorted = 10**profile.logT[ind_sort] 
    mu_sorted = profile.mu[ind_sort]

    # E = 3/2 * kT / (mu*m_p) 
    energy = np.trapz(3/2 * physical_constants.k * temp_sorted / (mu_sorted*physical_constants.m_p), x=mass_coord_sorted_g)
    return energy 



ages = [] 
energies = [] 
g_msquared_over_r = [] 
for modelnum in history.model_numbers_available: 
    print(f"{modelnum} / {np.max(history.model_numbers_available)}")
    profile = load_data.load_profile(stellar_evolution_data.data_folder/f"M={m}", modelnum=modelnum, history=history) 
    ages.append(history.star_age[modelnum-1]) 
    energies.append(calc_internal_energy(profile))
    g_msquared_over_r.append(physical_constants.G * (history.star_mass[modelnum-1]*physical_constants.M_sun)**2 / ((10**history.log_R[modelnum-1])*physical_constants.R_sun))




# Plot GM^2 / R 
plt.figure(figsize=(15, 8))
plt.plot(ages, g_msquared_over_r, label="GM^2 / R", lw=3) 
plt.plot(ages, energies, label="3/2*kT/(mu*m_p)", lw=3)
plt.yscale("log")
plt.xlabel("Age (years)")
plt.ylabel("ergs") 
title = f"Gravitational and Thermal Energy of a {m} M_sun star"
plt.title(title)
plt.grid(alpha=0.5) 
plt.legend() 
plt.savefig(title + ".jpg")



# Plot ratio 
plt.figure(figsize=(15, 8))
plt.plot(ages, np.array(energies)/np.array(g_msquared_over_r), lw=3) 
plt.ylim((0.5, 1.4))
plt.xlabel("Age (years)") 
title = f"Ratio of Thermal Energy to Gravitational Energy of a {m} M_sun star"
plt.title(title)
plt.grid(alpha=0.5) 
plt.savefig(title + ".jpg")







1 / 2550
50 / 2550
100 / 2550
150 / 2550
200 / 2550
250 / 2550
272 / 2550
300 / 2550
322 / 2550
350 / 2550
354 / 2550
400 / 2550
412 / 2550
419 / 2550
450 / 2550
494 / 2550
500 / 2550
550 / 2550
600 / 2550
650 / 2550
700 / 2550
750 / 2550
800 / 2550
850 / 2550
900 / 2550
950 / 2550
1000 / 2550
1050 / 2550
1100 / 2550
1150 / 2550
1200 / 2550
1250 / 2550
1300 / 2550
1350 / 2550
1400 / 2550
1450 / 2550
1500 / 2550
1550 / 2550
1600 / 2550
1650 / 2550
1700 / 2550
1750 / 2550
1800 / 2550
1850 / 2550
1900 / 2550
1950 / 2550
2000 / 2550
2050 / 2550
2100 / 2550
2150 / 2550
2200 / 2550
2250 / 2550
2300 / 2550
2350 / 2550
2400 / 2550
2450 / 2550
2500 / 2550
2550 / 2550


In [265]:
import matplotlib as mpl 

for n in range(12): 
    plt.plot([0+n,1+n], [0,1], color=mpl.colormaps["Set3"](n/12+1/24), lw=50)


